In [139]:
import numpy as np

In [226]:
data = np.random.randn(200, 15) + 3
mask = data<2.5
data[mask] = 0
data = np.round(data, 0)
data

array([[4., 0., 4., ..., 0., 0., 0.],
       [5., 4., 3., ..., 4., 0., 3.],
       [0., 5., 3., ..., 0., 4., 4.],
       ...,
       [4., 0., 0., ..., 3., 4., 5.],
       [3., 3., 3., ..., 3., 3., 0.],
       [4., 3., 5., ..., 4., 0., 0.]])

In [227]:
def cal_loss(data, P, Q):
    m, n = data.shape
    loss = 0.
    count = 0
    for i in range(m):
        for j in range(n):
            if data[i, j] !=0.:
                loss += np.power((data[i, j] - np.matmul(P[i,:], Q[:,j])),2)
                count += 1
    return np.round(loss / (0.01 + count))

# MF。 加减法更新权重

In [228]:
def up_date(data, k=2, maxiter=100, yuzhi=0.001, alpha=0.001, beta=0.5):
    # m 过大时 无法收敛？,填加 正则化项之后，无法收敛的。
    m, n =data.shape
    P = np.random.randn(m, k) + 2
    Q = np.random.randn(k, n) + 2
    err = data - np.matmul(P, Q)
    error_sum_old = np.sqrt(np.sum(err**2).sum())
    max_print = int(maxiter / 10)
    iters = 0
    while iters < maxiter:
        iters += 1
        old_P = P
        old_Q = Q # 可以不用的
        P = old_P + alpha * (np.matmul(err, old_Q.T) - beta * old_P)
        Q = old_Q + alpha * (np.matmul(old_P.T, err) - beta * old_Q)
        
        err = data - np.matmul(P, Q) # 这里讲 data[i,j]为0的直接不考虑了。算是一种近似吧
        #error_sum = np.sqrt(np.sum(err**2).sum())
        error_sum = cal_loss(data, P, Q)
        
        if iters % max_print == 0:
            print("error_sum is {}.".format(error_sum))
        if abs(error_sum_old - error_sum) < yuzhi:
            break
        else:
            error_sum_old = error_sum
    return P, Q, error_sum

In [230]:
def predicts(data, P, Q, user=0, top_k = 2):
    # P, Q, err_sum = up_date(data, maxiter=1000)
    m, n = data.shape
    no_point = []
    for index, value in enumerate(data[user]):
        if value == 0.0:
            no_point.append(index)
    if not no_point:
        return None
        # raise KeyError("Not need to recommend,,,")
    
    recond_dicts ={}
    for index in no_point:
        score = np.round(np.matmul(P[user,:], Q[:,index]),2)
        recond_dicts[index] = score
    
    recond_dicts = sorted(recond_dicts.items(), key=lambda row: row[1], reverse=True)
    if len(recond_dicts) < top_k:
        return recond_dicts
    else:
        return recond_dicts[:2]
    
P, Q, _ = up_date(data, maxiter=100)
predicts(data, P, Q, user=6)

[(2, 2.52), (6, 2.34)]

In [231]:
def recomd(data, front_k):
    m, _ = data.shape
    if m <= front_k or front_k < 0:
        raise KeyError("user must subject to [{}~{})".format(0, m))
    for i in range(front_k):
        result = predicts(data, P, Q, user=i)
        if result:
            print("recomd {}, {}".format(i, result))
        else:
            print("user_{}, no need to recommand".format(i))
            
recomd(data, 10)

recomd 0, [(1, 2.33), (12, 2.33)]
recomd 1, [(9, 2.24), (3, 2.14)]
recomd 2, [(12, 2.54), (9, 2.47)]
recomd 3, [(5, 0.91), (10, 0.83)]
recomd 4, [(1, 3.65), (2, 3.63)]
recomd 5, [(2, 3.18), (3, 2.99)]
recomd 6, [(2, 2.52), (6, 2.34)]
recomd 7, [(2, 2.05), (1, 1.98)]
recomd 8, [(2, 2.71), (12, 2.61)]
recomd 9, [(9, 2.87), (2, 2.86)]


In [ ]:
# FM 乘法更新权重

In [246]:
def cal_loss(data, P, Q):
    m, n = data.shape
    loss = 0.
    count = 0
    for i in range(m):
        for j in range(n):
            if data[i, j] !=0.:
                loss += np.power((data[i, j] - np.matmul(P[i,:], Q[:,j])),2)
                count += 1
    return np.round(loss / (0.01 + count))

def up_date_multiply(data, k=2, maxiter=100, yuzhi=0.001, alpha=0.001, beta=0.5):
    # m 过大时 无法收敛？,填加 正则化项之后，无法收敛的。
    m, n =data.shape
    P = np.random.random((m, k))
    Q = np.random.random((k, n))
    err = data - np.matmul(P, Q)
    error_sum_old = np.sqrt(np.sum(err**2).sum())
    max_print = int(maxiter / 10)
    iters = 0
    while iters < maxiter:
        iters += 1
        old_P = P
        old_Q = Q # 可以不用的
        # P = old_P + alpha * (np.matmul(err, old_Q.T) - beta * old_P)
        # Q = old_Q + alpha * (np.matmul(old_P.T, err) - beta * old_Q)
        # 需要注意这里的变换公式是什么
        P = np.multiply(old_P, np.matmul(data, old_Q.T) / np.matmul(old_P, np.matmul(old_Q, old_Q.T)))
        Q = np.multiply(old_Q, np.matmul(old_P.T, data) / np.matmul(old_P.T, np.matmul(old_P, old_Q)))
        
        err = data - np.matmul(P, Q) # 这里讲 data[i,j]为0的直接不考虑了。算是一种近似吧
        error_sum = np.sqrt(np.sum(err**2).sum())
        
        if iters % max_print == 0:
            print("error_sum is {}.".format(error_sum))
        #if abs(error_sum_old - error_sum) < yuzhi:
        #   break
        #else:
            #error_sum_old = error_sum
    return P, Q, error_sum

def predicts(data, P, Q, user=0, top_k = 2):
    m, n = data.shape
    no_point = []
    for index, value in enumerate(data[user]):
        if value == 0.0:
            no_point.append(index)
    if not no_point:
        raise KeyError("Not need to recommend,,,")
    recond_dicts ={}
    for index in no_point:
        score = np.round(np.matmul(P[user,:], Q[:,index]),2)
        recond_dicts[index] = score
    
    recond_dicts = sorted(recond_dicts.items(), key=lambda row: row[1], reverse=True)
    if len(recond_dicts) < top_k:
        return recond_dicts
    else:
        return recond_dicts[:2]
    
P, Q, _ = up_date_multiply(data, maxiter=8000)
predicts(data, P, Q, user=6)

error_sum is 137.17710358205917.
error_sum is 137.17710333241368.
error_sum is 137.17710333239904.
error_sum is 137.17710333239916.
error_sum is 137.17710333239927.
error_sum is 137.17710333239916.
error_sum is 137.1771033323989.
error_sum is 137.17710333239884.
error_sum is 137.17710333239887.
error_sum is 137.17710333239884.


[(2, 0.66), (11, 0.66)]

In [249]:
Q[Q<0]

array([], dtype=float64)

In [250]:
P[P<0]

array([], dtype=float64)